In [220]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

In [221]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

import pickle

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import chromedriver_binary


In [222]:
pd.set_option('display.max_columns', 100)
pd.set_option('max_colwidth', 1000)
pd.set_option('max_rows', 600)
requests.__path__

['/anaconda3/lib/python3.7/site-packages/requests']

In [223]:
url_lst = """https://stats.nba.com/players/drives/
https://stats.nba.com/players/catch-shoot/
https://stats.nba.com/players/passing/
https://stats.nba.com/players/touches/
https://stats.nba.com/players/pullup/
https://stats.nba.com/players/offensive-rebounding/
https://stats.nba.com/players/defensive-rebounding/
https://stats.nba.com/players/shooting-efficiency/
https://stats.nba.com/players/speed-distance/?sort=DIST_FEET&dir=1
https://stats.nba.com/players/defense-dash-overall/
https://stats.nba.com/players/shots-general/
https://stats.nba.com/players/shots-closest-defender/
https://stats.nba.com/players/shots-closest-defender/?Season=2018-19&SeasonType=Regular%20Season&CloseDefDistRange=2-4%20Feet%20-%20Tight
https://stats.nba.com/players/shots-closest-defender/?Season=2018-19&SeasonType=Regular%20Season&CloseDefDistRange=4-6%20Feet%20-%20Open
https://stats.nba.com/players/shots-closest-defender/?Season=2018-19&SeasonType=Regular%20Season&CloseDefDistRange=6%2B%20Feet%20-%20Wide%20Open
https://stats.nba.com/players/transition/
https://stats.nba.com/players/isolation/
https://stats.nba.com/players/ball-handler/
https://stats.nba.com/players/roll-man/
https://stats.nba.com/players/playtype-post-up/
https://stats.nba.com/players/spot-up/
https://stats.nba.com/players/hand-off/
https://stats.nba.com/players/cut/
https://stats.nba.com/players/off-screen/
https://stats.nba.com/players/putbacks/
https://stats.nba.com/players/playtype-misc/
https://stats.nba.com/players/isolation/?SeasonType=Regular%20Season&TypeGrouping=defensive
https://stats.nba.com/players/ball-handler/?SeasonType=Regular%20Season&TypeGrouping=defensive#!?SeasonType=Regular%20Season&TypeGrouping=defensive
https://stats.nba.com/players/playtype-post-up/?SeasonType=Regular%20Season&TypeGrouping=defensive
https://stats.nba.com/players/spot-up/?SeasonType=Regular%20Season&TypeGrouping=defensive
https://stats.nba.com/players/hand-off/?SeasonType=Regular%20Season&TypeGrouping=defensive
https://stats.nba.com/players/off-screen/?SeasonType=Regular%20Season&TypeGrouping=defensive
https://stats.nba.com/players/hustle/?sort=DEFLECTIONS&dir=1&Season=2018-19&SeasonType=Regular%20Season&PerMode=Per36
https://stats.nba.com/players/box-outs/?Season=2018-19&SeasonType=Regular%20Season&PerMode=Per36""".split('\n')

In [225]:
def get_data(url, sleep_val=8):
    driver = webdriver.Chrome()
    driver.get(url)
    data = []
    time.sleep(sleep_val)
    for _ in range(11):
        try: 
            page = driver.page_source
            soup = BeautifulSoup(page, 'html.parser')
            tr_list = soup.find_all('tr', href=False)
            for tr in tr_list[:]:
                td = tr.text.split('\n')
                data.append(td)
            next_button = driver.find_element_by_xpath("//a[@class='stats-table-pagination__next']")
            next_button.click()
        except:
            continue
    driver.close()
    return data

In [236]:
table_names = [url.split('/')[-2] for url in url_lst]
table_names = [name.replace('-','_') for name in table_names]
table_names

['drives',
 'catch_shoot',
 'passing',
 'touches',
 'pullup',
 'offensive_rebounding',
 'defensive_rebounding',
 'shooting_efficiency',
 'speed_distance',
 'defense_dash_overall',
 'shots_general',
 'shots_closest_defender',
 'shots_closest_defender',
 'shots_closest_defender',
 'shots_closest_defender',
 'transition',
 'isolation',
 'ball_handler',
 'roll_man',
 'playtype_post_up',
 'spot_up',
 'hand_off',
 'cut',
 'off_screen',
 'putbacks',
 'playtype_misc',
 'isolation',
 'ball_handler',
 'playtype_post_up',
 'spot_up',
 'hand_off',
 'off_screen',
 'hustle',
 'box_outs']

In [237]:
table_names[11] += '02'
table_names[12] += '24'
table_names[13] += '46'
table_names[14] += '6p'
table_names[26] = 'def_' + table_names[26]
table_names[27] = 'def_' + table_names[27]
table_names[28] = 'def_' + table_names[28]
table_names[29] = 'def_' + table_names[29]
table_names[30] = 'def_' + table_names[30]
table_names[31] = 'def_' + table_names[31]

In [235]:
data_dict = {table: get_data(url_lst[i]) for i, table in enumerate(table_names)}

MaxRetryError: HTTPConnectionPool(host='127.0.0.1', port=49461): Max retries exceeded with url: /session/d0a27692a98f24cfffc9c75660aaf238/window (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f8151ab37b8>: Failed to establish a new connection: [Errno 61] Connection refused'))

In [ ]:
def make_df(lst, col_name, col_pos_val=0, rename_col_val=3):
    column = [c.replace('\xa0','') for c in lst[col_pos_val]]
    df = pd.DataFrame(lst)
    df.columns = column
    df = df.iloc[1:,]
    column = [c for c in column if c != '']
    df = df[column]
    for col in column[rename_col_val:]:
        try:
            df[str(col)] = df[str(col)].str.strip('%')
            df[str(col)] = df[str(col)].str.strip(' ')
        except:
            continue
    df.columns = [col_name + ' ' + col if i > rename_col_val else col for i, col in enumerate(df.columns.to_list())]
    df = df.dropna()
    return df

In [ ]:
df_catch_shoot = make_df(data_dict['catch_shoot'], 'catch_shoot')
df_catch_shoot.head()

In [ ]:
df_drives = make_df(data_dict['drives'], 'drives', 2)

In [ ]:
df_passing = make_df(data_dict['passing'], 'passing')
df_passing.head()

In [ ]:
df_touches = make_df(data_dict["touches"], 'touches')
df_touches.head()

In [172]:
df_pullup = make_df(data_dict["pullup"], 'pullup')
df_pullup.head()

,Player,Team,GP,W,pullup L,pullup MIN,pullup PTS,pullup FGM,pullup FGA,pullup FG%,pullup 3PM,pullup 3PA,pullup 3P%,pullup eFG%
1,Aaron Gordon,ORL,78,40,38,33.8,2.8,1.2,3.4,36.2,0.4,1.2,32.3,41.9
2,Aaron Holiday,IND,50,31,19,12.9,2.1,0.9,2.6,33.8,0.4,1.2,29.5,40.8
3,Abdel Nader,OKC,61,38,23,11.4,0.2,0.1,0.2,30.8,0.0,0.1,40.0,38.5
4,Al Horford,BOS,68,41,27,29.0,0.6,0.3,0.7,45.8,0.0,0.0,-,45.8
5,Al-Farouq Aminu,POR,81,52,29,28.3,0.2,0.1,0.5,17.1,0.0,0.2,14.3,19.5


In [173]:
df_offensive_rebounding = make_df(data_dict["offensive_rebounding"], 'off_reb')
df_offensive_rebounding.head()

,Player,Team,GP,W,off_reb L,off_reb MIN,off_reb OREB,off_reb ContestedOREB,off_reb ContestedOREB%,off_reb OREBChances,off_reb OREBChance%,off_reb DeferredOREBChances,off_reb AdjustedOREBChance%,off_reb AVGOREBDistance
1,Aaron Gordon,ORL,76,38,38,33.8,1.7,1.0,60.2,3.6,47.2,0.1,49.2,7.2
2,Aaron Holiday,IND,50,31,19,12.9,0.1,0.0,40.0,0.4,27.8,0.0,27.8,16.0
3,Abdel Nader,OKC,60,37,23,11.4,0.2,0.1,21.4,0.7,31.8,0.0,31.8,16.8
4,Al Horford,BOS,68,41,27,29.0,1.8,0.9,49.2,3.9,45.5,0.1,46.2,8.2
5,Al-Farouq Aminu,POR,81,52,29,28.3,1.4,0.8,60.7,3.6,38.2,0.1,39.7,7.0


In [174]:
df_defensive_rebounding = make_df(data_dict["defensive_rebounding"], 'def_reb')
df_defensive_rebounding.head()

,Player,Team,GP,W,def_reb L,def_reb MIN,def_reb DREB,def_reb ContestedDREB,def_reb ContestedDREB%,def_reb DREBChances,def_reb DREBChance%,def_reb DeferredDREBChances,def_reb AdjustedDREBChance%,def_reb AVGDREBDistance
1,Aaron Gordon,ORL,76,38,38,33.8,5.8,1.5,25.7,8.2,70.8,0.6,76.0,6.3
2,Aaron Holiday,IND,50,31,19,12.9,1.2,0.1,8.1,2.0,63.3,0.2,72.1,9.0
3,Abdel Nader,OKC,60,37,23,11.4,1.7,0.4,21.0,2.6,63.3,0.2,67.6,7.3
4,Al Horford,BOS,68,41,27,29.0,5.0,1.7,34.6,8.9,55.7,0.8,61.2,5.8
5,Al-Farouq Aminu,POR,81,52,29,28.3,6.1,1.6,26.7,9.4,65.2,0.7,70.2,5.2


In [175]:
df_shooting_efficiency = make_df(data_dict["shooting_efficiency"], 'shot_eff')
df_shooting_efficiency.head()

,Player,Team,GP,W,shot_eff L,shot_eff MIN,shot_eff PTS,shot_eff DrivePTS,shot_eff DriveFG%,shot_eff C&SPTS,shot_eff C&SFG%,shot_eff PullUpPTS,shot_eff PullUpFG%,shot_eff PaintTouchPTS,shot_eff PaintTouchFG%,shot_eff PostTouchPTS,shot_eff PostTouchFG%,shot_eff ElbowTouchPTS,shot_eff ElbowTouchFG%,shot_eff eFG%
1,Aaron Gordon,ORL,76,38,38,33.8,16.2,3.2,42.9,3.9,35.5,2.9,36.2,3.9,64.5,1.5,40.4,1.1,50.7,51.0
2,Aaron Holiday,IND,50,31,19,12.9,5.9,1.4,42.3,1.5,37.9,2.1,33.8,0.0,100,0.0,0.0,0.2,100,48.3
3,Abdel Nader,OKC,60,37,23,11.4,4.0,1.4,50.7,1.5,30.5,0.2,30.8,0.3,57.1,0.0,0.0,0.2,42.9,49.5
4,Al Horford,BOS,68,41,27,29.0,13.6,1.1,56.0,5.8,44.4,0.6,45.8,4.1,70.6,2.0,51.4,1.5,60.3,58.6
5,Al-Farouq Aminu,POR,81,52,29,28.3,9.4,1.8,46.5,3.6,35.1,0.2,17.1,2.0,61.9,0.1,42.9,0.4,45.0,51.4


In [176]:
df_speed_distance = make_df(data_dict["speed_distance"], 'speed_dist')
df_speed_distance.head()

,Player,Team,GP,W,speed_dist L,speed_dist MIN,speed_dist Dist.Feet,speed_dist Dist.Miles,speed_dist Dist.MilesOff,speed_dist Dist.MilesDef,speed_dist AvgSpeed,speed_dist AvgSpeedOff,speed_dist AvgSpeedDef
1,Jrue Holiday,NOP,67,30,37,35.9,14546.4,2.75,1.51,1.25,4.32,4.92,3.76
2,Bradley Beal,WAS,81,31,50,36.9,14516.8,2.75,1.46,1.29,4.19,4.53,3.85
3,Paul George,OKC,76,45,31,36.9,14221.9,2.69,1.40,1.29,4.09,4.59,3.65
4,Zach LaVine,CHI,62,16,46,34.4,14211.2,2.69,1.48,1.21,4.43,4.87,3.99
5,Josh Richardson,MIA,73,35,38,34.8,14120.4,2.67,1.50,1.17,4.30,4.72,3.85


In [177]:
df_defense = make_df(data_dict["defense_dash_overall"], 'defense')
df_defense.head()

,Player,Team,Age,Position,defense GP,defense G,defense FREQ,defense DFGM,defense DFGA,defense DFG%,defense FG%,defense Diff%
1,Joel Embiid,PHI,25,F-C,63,63,100.0,8.4,18.6,45.0,48.1,-3.1
2,Rudy Gobert,UTA,27,C,80,80,100.0,8.0,17.8,45.0,48.9,-3.9
3,Clint Capela,HOU,25,C,67,67,100.0,8.3,17.7,46.7,47.1,-0.4
4,Steven Adams,OKC,25,C,79,79,100.0,7.8,16.4,47.6,48.4,-0.8
5,Brook Lopez,MIL,31,C,81,81,100.0,7.3,16.2,44.9,48.7,-3.8


In [178]:
df_shots_general = make_df(data_dict["shots_general"], 'shots_general', col_pos_val=1)

In [179]:
df_shots_closest_defender02 = make_df(data_dict["shots_closest_defender02"], 'defender02ft', col_pos_val=1, rename_col_val=5)
df_shots_closest_defender02.head()

,Player,Team,Age,GP,G,Freq,defender02ft FGM,defender02ft FGA,defender02ft FG%,defender02ft eFG%,defender02ft 2FG Freq,defender02ft 2FGM,defender02ft 2FGA,defender02ft 2FG%,defender02ft 3FG Freq,defender02ft 3PM,defender02ft 3PA,defender02ft 3P%
1,Player,Team,Age,GP,G,Freq,FGM,FGA,FG,eFG,2FG Freq,2FGM,2FGA,2FG,3FG Freq,3PM,3PA,3P
2,Blake Griffin,DET,30,75,71,19.5,1.8,3.5,50.2,50.8,18.9,1.7,3.4,50.6,0.6,0.0,0.1,37.5
3,Joel Embiid,PHI,25,63,61,18.2,1.5,3.4,42.8,42.8,18.1,1.5,3.4,43.2,0.2,0.0,0.0,0.0
4,Giannis Antetokounmpo,MIL,24,72,68,19.5,1.9,3.4,57.6,57.6,19.4,1.9,3.4,57.9,0.1,0.0,0.0,0.0
5,Anthony Davis,NOP,26,56,51,18.4,1.8,3.4,54.0,54.0,18.2,1.8,3.3,54.5,0.2,0.0,0.0,0.0


In [180]:
df_shots_closest_defender24 = make_df(data_dict["shots_closest_defender24"], 'defender24ft', col_pos_val=1)
df_shots_closest_defender24.head()

,Player,Team,Age,GP,defender24ft G,defender24ft Freq,defender24ft FGM,defender24ft FGA,defender24ft FG%,defender24ft eFG%,defender24ft 2FG Freq,defender24ft 2FGM,defender24ft 2FGA,defender24ft 2FG%,defender24ft 3FG Freq,defender24ft 3PM,defender24ft 3PA,defender24ft 3P%
1,Player,Team,Age,GP,G,Freq,FGM,FGA,FG,eFG,2FG Freq,2FGM,2FGA,2FG,3FG Freq,3PM,3PA,3P
2,James Harden,HOU,29,78,78,53.4,5.9,13.1,45.1,52.3,29.9,4.0,7.3,55.1,23.6,1.9,5.8,32.4
3,Donovan Mitchell,UTA,22,76,76,46.2,3.9,9.1,43.3,44.7,41.3,3.7,8.1,45.4,4.9,0.3,1.0,26.0
4,Devin Booker,PHX,22,64,64,44.2,4.2,8.7,48.6,51.4,36.8,3.7,7.2,51.5,7.3,0.5,1.4,33.7
5,Kevin Durant,GSW,30,78,77,48.0,4.3,8.4,51.3,53.6,39.3,3.9,6.9,56.9,8.7,0.4,1.5,26.1


In [181]:
df_shots_closest_defender46 = make_df(data_dict["shots_closest_defender46"], 'defender46ft', col_pos_val=1)
df_shots_closest_defender46.head()

,Player,Team,Age,GP,defender46ft G,defender46ft Freq,defender46ft FGM,defender46ft FGA,defender46ft FG%,defender46ft eFG%,defender46ft 2FG Freq,defender46ft 2FGM,defender46ft 2FGA,defender46ft 2FG%,defender46ft 3FG Freq,defender46ft 3PM,defender46ft 3PA,defender46ft 3P%
1,Player,Team,Age,GP,G,Freq,FGM,FGA,FG,eFG,2FG Freq,2FGM,2FGA,2FG,3FG Freq,3PM,3PA,3P
2,Paul George,OKC,29,76,76,37.9,3.1,7.9,39.5,49.3,15.4,1.6,3.2,48.6,22.5,1.6,4.7,33.2
3,James Harden,HOU,29,78,78,31.4,3.4,7.7,43.7,59.0,6.9,1.0,1.7,59.1,24.5,2.4,6.0,39.3
4,Kawhi Leonard,TOR,28,60,60,37.7,3.3,7.1,46.5,53.5,22.9,2.3,4.3,53.5,14.9,1.0,2.8,35.7
5,Stephen Curry,GSW,31,69,69,36.8,3.0,7.0,43.0,57.5,9.4,1.0,1.8,54.8,27.4,2.0,5.3,39.0


In [182]:
df_shots_closest_defender6p = make_df(data_dict["shots_closest_defender6p"], 'defender6pft', col_pos_val=1)
df_shots_closest_defender6p.head()

,Player,Team,Age,GP,defender6pft G,defender6pft Freq,defender6pft FGM,defender6pft FGA,defender6pft FG%,defender6pft eFG%,defender6pft 2FG Freq,defender6pft 2FGM,defender6pft 2FGA,defender6pft 2FG%,defender6pft 3FG Freq,defender6pft 3PM,defender6pft 3PA,defender6pft 3P%
1,Player,Team,Age,GP,G,Freq,FGM,FGA,FG,eFG,2FG Freq,2FGM,2FGA,2FG,3FG Freq,3PM,3PA,3P
2,RJ Hunter,BOS,25,1,1,46.2,4.0,6.0,66.7,100,0.0,0.0,0.0,-,46.2,4.0,6.0,66.7
3,Eric Gordon,HOU,30,68,68,40.8,2.3,5.6,41.6,61.8,1.2,0.1,0.2,45.5,39.6,2.3,5.5,41.5
4,Buddy Hield,SAC,26,82,81,30.6,2.6,5.1,50.7,71.5,4.7,0.5,0.8,59.4,25.9,2.1,4.3,49.1
5,Stephen Curry,GSW,31,69,68,24.7,2.5,4.7,52.8,75.2,3.6,0.4,0.7,54.2,21.1,2.1,4.0,52.5


In [189]:
df_off_transition = make_df(data_dict["transition"], "offense_transition", rename_col_val=3)
df_off_transition.head()

,Player,Team,GP,Poss,offense_transition Freq,offense_transition PPP,offense_transition PTS,offense_transition FGM,offense_transition FGA,offense_transition FG%,offense_transition eFG%,offense_transition FTFreq,offense_transition TOVFreq,offense_transition SFFreq,offense_transition AndOneFreq,offense_transition ScoreFreq,offense_transition Percentile
1,Giannis Antetokounmpo,MIL,72,6.4,25.6,1.14,7.3,1.4,3.9,64.0,64.9,28.6,16.9,26.6,6.3,58.7,55.8
2,Buddy Hield,SAC,82,5.0,26.1,1.21,6.0,2.1,4.4,51.0,62.0,7.5,7.3,7.3,2.2,49.9,70.2
3,Bradley Beal,WAS,82,5.1,20.9,1.14,5.8,2.1,4.1,48.8,59.0,11.7,9.3,10.3,1.7,49.2,55.6
4,Paul George,OKC,76,5.4,20.6,1.11,6.0,2.3,4.4,48.2,56.7,12.1,9.7,9.7,2.9,47.8,49.6
5,Kevin Durant,GSW,78,4.9,21.1,1.17,5.8,1.4,3.5,58.5,62.9,18.1,15.0,16.8,3.9,54.7,63.4


In [190]:
df_off_isolation = make_df(data_dict["isolation"], "offense_isolation", rename_col_val=3)
df_off_isolation.head()

,Player,Team,GP,Poss,offense_isolation Freq,offense_isolation PPP,offense_isolation PTS,offense_isolation FGM,offense_isolation FGA,offense_isolation FG%,offense_isolation eFG%,offense_isolation FTFreq,offense_isolation TOVFreq,offense_isolation SFFreq,offense_isolation AndOneFreq,offense_isolation ScoreFreq,offense_isolation Percentile
1,Christian Wood,NOP,8,1.3,12.3,0.20,0.3,1.1,1.3,10.0,10.0,0.0,0.0,0.0,0.0,10.0,100
2,Khem Birch,ORL,50,0.2,3.4,0.25,0.1,0.2,0.2,0.0,0.0,17.0,8.0,17.0,0.0,17.0,99.8
3,Deyonta Davis,ATL,9,1.1,13.9,0.60,0.7,0.7,0.9,25.0,25.0,10.0,10.0,10.0,0.0,30.0,93.6
4,Brandon Knight,CLE,27,0.4,5.0,0.60,0.2,0.2,0.3,29.0,29.0,20.0,10.0,20.0,0.0,40.0,93.6
5,Dillon Brooks,MEM,18,0.6,8.4,0.60,0.3,0.3,0.4,14.0,14.0,20.0,10.0,20.0,0.0,30.0,93.6


In [191]:
df_off_ball_handler = make_df(data_dict["ball_handler"], "offense_ball_handler", rename_col_val=3)
df_off_ball_handler.head()

,Player,Team,GP,Poss,offense_ball_handler Freq,offense_ball_handler PPP,offense_ball_handler PTS,offense_ball_handler FGM,offense_ball_handler FGA,offense_ball_handler FG%,offense_ball_handler eFG%,offense_ball_handler FTFreq,offense_ball_handler TOVFreq,offense_ball_handler SFFreq,offense_ball_handler AndOneFreq,offense_ball_handler ScoreFreq,offense_ball_handler Percentile
1,Ron Baker,WAS,4,3.0,75.0,0.33,1.0,1.8,2.3,22.2,22.2,0.0,25.0,0.0,0.0,16.7,100
2,Al Horford,BOS,68,0.0,1.0,0.50,0.0,0.1,0.1,22.2,22.2,10.0,10.0,10.0,10.0,20.0,98.8
3,Myles Turner,IND,74,0.0,1.0,0.36,0.0,0.1,0.1,20.0,20.0,7.0,21.0,7.0,0.0,21.4,99.5
4,Steven Adams,OKC,79,0.0,1.0,0.54,0.0,0.1,0.1,33.3,38.9,0.0,31.0,0.0,0.0,23.1,98.3
5,Rudy Gobert,UTA,81,0.0,1.0,0.73,0.0,0.1,0.1,33.3,33.3,18.0,9.0,18.0,9.0,36.4,88.9


In [192]:
df_off_roll_man = make_df(data_dict["roll_man"], "offense_roll_man", rename_col_val=3)
df_off_roll_man.head()

,Player,Team,GP,Poss,offense_roll_man Freq,offense_roll_man PPP,offense_roll_man PTS,offense_roll_man FGM,offense_roll_man FGA,offense_roll_man FG%,offense_roll_man eFG%,offense_roll_man FTFreq,offense_roll_man TOVFreq,offense_roll_man SFFreq,offense_roll_man AndOneFreq,offense_roll_man ScoreFreq,offense_roll_man Percentile
1,Nikola Vucevic,ORL,80,4.1,20.5,1.13,4.7,1.6,3.6,54.2,58.5,6.9,7.3,6.9,1.2,52.9,61.1
2,Rudy Gobert,UTA,81,3.4,25.3,1.35,4.5,0.7,2.4,70.9,70.9,28.3,6.3,26.8,7.0,69.1,93.1
3,Montrezl Harrell,LAC,82,3.4,23.3,1.31,4.4,0.8,2.7,68.9,69.2,20.5,8.3,20.1,7.6,66.2,87.5
4,Myles Turner,IND,74,4.7,35.9,1.03,4.9,2.1,4.1,48.2,52.6,9.5,6.3,9.2,2.6,48.1,41.7
5,Serge Ibaka,TOR,74,4.3,29.7,1.10,4.7,1.7,3.8,54.6,55.9,8.5,4.4,8.2,1.3,55.2,56.5


In [193]:
df_off_playtype_post_up = make_df(data_dict["playtype_post_up"], "offense_playtype_post_up", rename_col_val=3)
df_off_playtype_post_up.head()

,Player,Team,GP,Poss,offense_playtype_post_up Freq,offense_playtype_post_up PPP,offense_playtype_post_up PTS,offense_playtype_post_up FGM,offense_playtype_post_up FGA,offense_playtype_post_up FG%,offense_playtype_post_up eFG%,offense_playtype_post_up FTFreq,offense_playtype_post_up TOVFreq,offense_playtype_post_up SFFreq,offense_playtype_post_up AndOneFreq,offense_playtype_post_up ScoreFreq,offense_playtype_post_up Percentile
1,Omari Spellman,ATL,45,0.2,3.1,0.20,0.0,0.2,0.2,12.5,12.5,0.0,20.0,0.0,0.0,10.0,100
2,CJ Miles,TOR,40,0.3,7.9,0.40,0.1,0.1,0.2,28.6,28.6,10.0,20.0,10.0,0.0,20.0,99.0
3,Khris Middleton,MIL,77,0.2,1.7,0.33,0.1,0.1,0.1,14.3,14.3,25.0,17.0,25.0,0.0,25.0,99.5
4,George Hill,MIL,47,0.3,3.3,0.31,0.1,0.2,0.2,20.0,20.0,0.0,23.0,0.0,0.0,15.0,99.8
5,Robert Covington,PHI,13,0.8,5.5,0.46,0.4,0.4,0.5,28.6,28.6,9.0,27.0,9.0,0.0,27.0,98.3


In [197]:
df_off_spot_up = make_df(data_dict["spot_up"], "offense_spot_up", rename_col_val=3)
df_off_spot_up.head()

,Player,Team,GP,Poss,offense_spot_up Freq,offense_spot_up PPP,offense_spot_up PTS,offense_spot_up FGM,offense_spot_up FGA,offense_spot_up FG%,offense_spot_up eFG%,offense_spot_up FTFreq,offense_spot_up TOVFreq,offense_spot_up SFFreq,offense_spot_up AndOneFreq,offense_spot_up ScoreFreq,offense_spot_up Percentile
1,Quincy Acy,PHX,10,1.0,16.4,0.30,0.3,0.6,0.7,14.3,21.4,10.0,20.0,10.0,0.0,10.0,99.8
2,Isaiah Thomas,DEN,12,1.0,21.6,0.55,0.5,0.8,0.8,10.0,15.0,9.0,0.0,9.0,0.0,18.0,99.2
3,Stanley Johnson,NOP,18,1.0,17.1,0.69,0.5,0.4,0.6,30.0,45.0,0.0,23.0,0.0,0.0,23.0,95.9
4,Andrew Bogut,GSW,11,1.0,16.7,0.69,0.8,0.7,1.0,27.3,31.8,8.0,8.0,0.0,0.0,31.0,95.9
5,Jonathon Simmons,PHI,15,1.0,24.2,0.60,0.6,0.6,0.9,30.8,30.8,7.0,7.0,0.0,0.0,33.0,98.5


In [198]:
df_off_hand_off = make_df(data_dict["hand_off"], "offense_hand_off", rename_col_val=3)
df_off_hand_off.head()

,Player,Team,GP,Poss,offense_hand_off Freq,offense_hand_off PPP,offense_hand_off PTS,offense_hand_off FGM,offense_hand_off FGA,offense_hand_off FG%,offense_hand_off eFG%,offense_hand_off FTFreq,offense_hand_off TOVFreq,offense_hand_off SFFreq,offense_hand_off AndOneFreq,offense_hand_off ScoreFreq,offense_hand_off Percentile
1,Alex Caruso,LAL,26,0.5,5.2,0.17,0.1,0.3,0.4,10.0,10.0,0.0,16.7,0.0,0.0,8.3,100
2,Al Horford,BOS,68,0.1,1.1,0.50,0.1,0.1,0.1,0.0,0.0,20.0,0.0,20.0,0.0,20.0,96.5
3,Semi Ojeleye,BOS,56,0.2,5.4,0.46,0.1,0.1,0.2,20.0,25.0,0.0,9.1,0.0,0.0,18.2,98.3
4,Thabo Sefolosha,UTA,50,0.2,5.6,0.46,0.1,0.2,0.2,20.0,25.0,0.0,9.1,0.0,0.0,18.2,98.3
5,Iman Shumpert,HOU,20,0.6,8.9,0.46,0.3,0.4,0.5,20.0,25.0,0.0,9.1,0.0,0.0,18.2,98.3


In [199]:
df_off_cut = make_df(data_dict["cut"], "offense_cut", rename_col_val=3)
df_off_cut.head()

,Player,Team,GP,Poss,offense_cut Freq,offense_cut PPP,offense_cut PTS,offense_cut FGM,offense_cut FGA,offense_cut FG%,offense_cut eFG%,offense_cut FTFreq,offense_cut TOVFreq,offense_cut SFFreq,offense_cut AndOneFreq,offense_cut ScoreFreq,offense_cut Percentile
1,Rudy Gobert,UTA,81,3.5,26.4,1.29,4.5,0.8,2.6,69.2,69.2,22.9,6.0,23.2,4.6,68.3,52.3
2,Clint Capela,HOU,67,4.0,28.3,1.35,5.3,1.1,3.5,69.5,69.5,11.7,1.9,11.7,3.0,68.7,64.6
3,JaVale McGee,LAL,75,2.9,26.5,1.38,4.0,0.7,2.5,71.4,71.4,13.0,5.1,12.5,3.7,69.4,72.7
4,Montrezl Harrell,LAC,82,2.5,17.2,1.42,3.5,0.6,2.0,71.9,71.9,23.4,4.9,23.4,9.8,69.8,76.7
5,Ben Simmons,PHI,79,2.7,15.1,1.30,3.5,0.7,2.3,68.9,68.9,12.6,6.5,12.1,2.8,66.5,53.9


In [202]:
df_off_off_screen = make_df(data_dict["off_screen"], "offense_off_screen", rename_col_val=3)
df_off_off_screen.head()

,Player,Team,GP,Poss,offense_off_screen Freq,offense_off_screen PPP,offense_off_screen PTS,offense_off_screen FGM,offense_off_screen FGA,offense_off_screen FG%,offense_off_screen eFG%,offense_off_screen FTFreq,offense_off_screen TOVFreq,offense_off_screen SFFreq,offense_off_screen AndOneFreq,offense_off_screen ScoreFreq,offense_off_screen Percentile
1,Jaylen Adams,ATL,33,0.3,7.5,0.50,0.2,0.2,0.3,11.1,11.1,10.0,0.0,10.0,0.0,20.0,98.9
2,Jaron Blossomgame,CLE,28,0.4,7.9,0.50,0.2,0.3,0.3,12.5,18.8,10.0,10.0,10.0,0.0,20.0,98.9
3,Nikola Vucevic,ORL,80,0.1,0.8,0.60,0.1,0.1,0.1,12.5,12.5,20.0,0.0,20.0,0.0,30.0,93.8
4,Dragan Bender,PHX,46,0.2,2.5,0.70,0.2,0.2,0.2,30.0,35.0,0.0,0.0,0.0,0.0,30.0,86.2
5,Svi Mykhailiuk,LAL,39,0.3,6.9,0.70,0.2,0.2,0.3,30.0,35.0,0.0,0.0,0.0,0.0,30.0,86.2


In [204]:
df_off_putbacks = make_df(data_dict["putbacks"], "offense_putbacks", rename_col_val=3)
df_off_putbacks.head()

,Player,Team,GP,Poss,offense_putbacks Freq,offense_putbacks PPP,offense_putbacks PTS,offense_putbacks FGM,offense_putbacks FGA,offense_putbacks FG%,offense_putbacks eFG%,offense_putbacks FTFreq,offense_putbacks TOVFreq,offense_putbacks SFFreq,offense_putbacks AndOneFreq,offense_putbacks ScoreFreq,offense_putbacks Percentile
1,Andre Drummond,DET,79,4.6,25.8,1.05,4.9,1.8,4.0,53.4,53.4,13.1,3.8,12.3,2.5,54.1,45.6
2,Rudy Gobert,UTA,81,2.8,20.9,1.23,3.4,0.8,2.2,62.4,62.4,17.3,4.0,16.0,1.8,64.0,77.8
3,Hassan Whiteside,MIA,72,3.1,25.8,1.07,3.3,1.3,2.8,55.0,55.0,12.4,1.8,12.0,4.0,54.2,50.3
4,Karl-Anthony Towns,MIN,77,2.5,11.2,1.19,3.0,0.8,2.0,61.1,62.1,13.3,9.7,13.3,3.6,59.0,72.5
5,LaMarcus Aldridge,SAS,81,2.1,10.2,1.19,2.5,0.7,1.7,58.6,58.6,13.7,6.5,13.7,3.6,58.9,71.3


In [205]:
df_off_putbacks = make_df(data_dict["playtype_misc"], "offense_playtype_misc", rename_col_val=3)
df_off_putbacks.head()

,Player,Team,GP,Poss,offense_playtype_misc Freq,offense_playtype_misc PPP,offense_playtype_misc PTS,offense_playtype_misc FGM,offense_playtype_misc FGA,offense_playtype_misc FG%,offense_playtype_misc eFG%,offense_playtype_misc FTFreq,offense_playtype_misc TOVFreq,offense_playtype_misc SFFreq,offense_playtype_misc AndOneFreq,offense_playtype_misc ScoreFreq,offense_playtype_misc Percentile
1,Blake Griffin,DET,75,1.7,7.0,0.71,1.2,0.3,0.5,45.0,52.5,26.6,43.8,3.1,1.6,35.9,78.9
2,James Harden,HOU,78,1.6,4.6,0.73,1.1,0.3,0.5,39.0,45.8,27.0,45.1,9.0,1.6,36.1,80.5
3,Russell Westbrook,OKC,72,1.6,5.7,0.79,1.2,0.3,0.7,52.0,58.3,20.5,37.5,5.4,0.9,38.4,86.4
4,Joel Embiid,PHI,64,2.2,8.1,0.61,1.3,0.3,0.4,28.0,30.0,32.1,50.7,3.6,0.7,35.7,62.9
5,Nikola Jokic,DEN,80,1.9,9.3,0.56,1.1,0.1,0.2,42.0,47.4,26.8,61.7,3.4,1.3,30.2,55.5


In [206]:
df_off_misc = make_df(data_dict[ 'playtype_misc'], 'off_misc')

In [217]:
df_def_isolation = make_df(data_dict["def_isolation"], "def_isolation", rename_col_val=4)
df_def_isolation.head()

NameError: name 'def_playtype_dict' is not defined

In [218]:
df_def_ball_handler = make_df(data_dict["ball_handler_def"], "def_ball_handler", rename_col_val=4)
df_def_ball_handler.head()

NameError: name 'def_playtype_dict' is not defined

In [353]:
df_def_playtype_post_up = make_df(data_dict["def_playtype_post_up"], "def_playtype_post_up", rename_col_val=4)
df_def_playtype_post_up.head()

,Player,Team,GP,Poss,Freq,def_playtype_post_up PPP,def_playtype_post_up PTS,def_playtype_post_up FGM,def_playtype_post_up FGA,def_playtype_post_up FG%,def_playtype_post_up eFG%,def_playtype_post_up FTFreq,def_playtype_post_up TOVFreq,def_playtype_post_up SFFreq,def_playtype_post_up AndOneFreq,def_playtype_post_up ScoreFreq,def_playtype_post_up Percentile
1,Omari Spellman,ATL,45,0.2,3.1,0.20,0.0,0.2,0.2,12.5,12.5,0.0,20.0,0.0,0.0,10.0,100
2,CJ Miles,TOR,40,0.3,7.9,0.40,0.1,0.1,0.2,28.6,28.6,10.0,20.0,10.0,0.0,20.0,99.0
3,Khris Middleton,MIL,77,0.2,1.7,0.33,0.1,0.1,0.1,14.3,14.3,25.0,17.0,25.0,0.0,25.0,99.5
4,George Hill,MIL,47,0.3,3.3,0.31,0.1,0.2,0.2,20.0,20.0,0.0,23.0,0.0,0.0,15.0,99.8
5,Robert Covington,PHI,13,0.8,5.5,0.46,0.4,0.4,0.5,28.6,28.6,9.0,27.0,9.0,0.0,27.0,98.3


In [354]:
df_def_spot_up = make_df(data_dict["def_spot_up"], "def_spot_up", rename_col_val=4)
df_def_spot_up.head()

,Player,Team,GP,Poss,Freq,def_spot_up PPP,def_spot_up PTS,def_spot_up FGM,def_spot_up FGA,def_spot_up FG%,def_spot_up eFG%,def_spot_up FTFreq,def_spot_up TOVFreq,def_spot_up SFFreq,def_spot_up AndOneFreq,def_spot_up ScoreFreq,def_spot_up Percentile
1,Quincy Acy,PHX,10,1.0,16.4,0.30,0.3,0.6,0.7,14.3,21.4,10.0,20.0,10.0,0.0,10.0,99.8
2,Isaiah Thomas,DEN,12,1.0,21.6,0.55,0.5,0.8,0.8,10.0,15.0,9.0,0.0,9.0,0.0,18.0,99.2
3,Stanley Johnson,NOP,18,1.0,17.1,0.69,0.5,0.4,0.6,30.0,45.0,0.0,23.0,0.0,0.0,23.0,95.9
4,Andrew Bogut,GSW,11,1.0,16.7,0.69,0.8,0.7,1.0,27.3,31.8,8.0,8.0,0.0,0.0,31.0,95.9
5,Jonathon Simmons,PHI,15,1.0,24.2,0.60,0.6,0.6,0.9,30.8,30.8,7.0,7.0,0.0,0.0,33.0,98.5


In [355]:
df_def_hand_off = make_df(data_dict["def_hand_off_def"], "def_hand_off", rename_col_val=4)
df_def_hand_off.head()

,Player,Team,GP,Poss,Freq,def_hand_off PPP,def_hand_off PTS,def_hand_off FGM,def_hand_off FGA,def_hand_off FG%,def_hand_off eFG%,def_hand_off FTFreq,def_hand_off TOVFreq,def_hand_off SFFreq,def_hand_off AndOneFreq,def_hand_off ScoreFreq,def_hand_off Percentile
1,Alex Caruso,LAL,26,0.5,5.2,0.17,0.1,0.3,0.4,10.0,10.0,0.0,16.7,0.0,0.0,8.3,100
2,Al Horford,BOS,68,0.1,1.1,0.50,0.1,0.1,0.1,0.0,0.0,20.0,0.0,20.0,0.0,20.0,96.5
3,Semi Ojeleye,BOS,56,0.2,5.4,0.46,0.1,0.1,0.2,20.0,25.0,0.0,9.1,0.0,0.0,18.2,98.3
4,Thabo Sefolosha,UTA,50,0.2,5.6,0.46,0.1,0.2,0.2,20.0,25.0,0.0,9.1,0.0,0.0,18.2,98.3
5,Iman Shumpert,HOU,20,0.6,8.9,0.46,0.3,0.4,0.5,20.0,25.0,0.0,9.1,0.0,0.0,18.2,98.3


In [356]:
df_def_off_screen = make_df(data_dict["def_off_screen"], "def_off_screen", rename_col_val=4)
df_def_off_screen.head()

,Player,Team,GP,Poss,Freq,def_off_screen PPP,def_off_screen PTS,def_off_screen FGM,def_off_screen FGA,def_off_screen FG%,def_off_screen eFG%,def_off_screen FTFreq,def_off_screen TOVFreq,def_off_screen SFFreq,def_off_screen AndOneFreq,def_off_screen ScoreFreq,def_off_screen Percentile
1,Jaylen Adams,ATL,33,0.3,7.5,0.50,0.2,0.2,0.3,11.1,11.1,10.0,0.0,10.0,0.0,20.0,98.9
2,Jaron Blossomgame,CLE,28,0.4,7.9,0.50,0.2,0.3,0.3,12.5,18.8,10.0,10.0,10.0,0.0,20.0,98.9
3,Nikola Vucevic,ORL,80,0.1,0.8,0.60,0.1,0.1,0.1,12.5,12.5,20.0,0.0,20.0,0.0,30.0,93.8
4,Dragan Bender,PHX,46,0.2,2.5,0.70,0.2,0.2,0.2,30.0,35.0,0.0,0.0,0.0,0.0,30.0,86.2
5,Svi Mykhailiuk,LAL,39,0.3,6.9,0.70,0.2,0.2,0.3,30.0,35.0,0.0,0.0,0.0,0.0,30.0,86.2


In [38]:
df_hustle = make_df(data_dict["hustle"], "hustle", rename_col_val=4)

In [42]:
df_deflect = make_df(data_dict["box_outs"], "box_outs", rename_col_val=5)

In [43]:
df_deflect

,Player,TEAM,AGE,GP,MIN,Box Outs,box_outs OFF Box Outs,box_outs DEF Box Outs,box_outs Team RebOn Box Outs,box_outs Player RebOn Box Outs,box_outs % Box Outs Off,box_outs % Box Outs Def,box_outs % Team RebWhen Box Out,box_outs % Player RebWhen Box Out
1,Aaron Gordon,ORL,23,78,2633,2.6,0.2,2.3,1.2,0.4,9.1,90.9,85.6,28.8
2,Aaron Holiday,IND,22,50,646,1.3,0.0,1.3,0.7,0.0,0.0,100,92.9,0.0
3,Abdel Nader,OKC,25,61,694,2.5,0.0,2.5,1.4,0.5,0.0,100,79.4,29.4
4,Al Horford,BOS,33,68,1973,8.3,0.6,7.7,4.1,1.4,7.3,92.7,86.1,30.1
5,Al-Farouq Aminu,POR,28,81,2292,4.5,0.4,4.2,2.1,0.9,8.0,92.0,87.5,37.5
6,Alan Williams,BKN,26,5,26,5.5,1.4,4.2,0.0,0.0,25.0,75.0,0.0,0.0
7,Alec Burks,SAC,27,64,1375,0.8,0.0,0.8,0.5,0.1,0.0,100,95.0,20.0
8,Alex Abrines,OKC,25,31,588,2.7,0.0,2.7,1.5,0.2,0.0,100,75.0,9.4
9,Alex Caruso,LAL,25,25,531,2.0,0.0,2.0,1.3,0.3,0.0,100,86.4,18.2
10,Alex Len,ATL,26,77,1544,10.3,1.4,8.9,5.3,1.6,13.4,86.6,89.3,26.5


In [380]:
df_drives.to_pickle('./data/df_drives.pkl')
df_catch_shoot.to_pickle('./data/df_catch_shoot.pkl')
df_passing.to_pickle('./data/df_passing.pkl')
df_touches.to_pickle('./data/df_touches.pkl')
df_pullup.to_pickle('./data/df_pullup.pkl')
df_offensive_rebounding.to_pickle('./data/df_off_reb.pkl')
df_defensive_rebounding.to_pickle('./data/df_def_reb.pkl')
df_shooting_efficiency.to_pickle('./data/df_shot_eff.pkl')
df_speed_distance.to_pickle('./data/df_speed_dist.pkl')
df_defense.to_pickle('./data/df_defense.pkl')
df_shots_general.to_pickle('./data/df_shots_general.pkl')
df_shots_closest_defender02.to_pickle('./data/df_defender_02ft.pkl')
df_shots_closest_defender24.to_pickle('./data/df_defender_24ft.pkl')
df_shots_closest_defender46.to_pickle('./data/df_defender_46ft.pkl')
df_shots_closest_defender6p.to_pickle('./data/df_defender_6pft.pkl')
df_off_transition.to_pickle('./data/df_off_transition.pkl')
df_off_isolation.to_pickle('./data/df_off_iso.pkl')
df_off_ball_handler.to_pickle('./data/df_off_ball_handler.pkl')
df_off_roll_man.to_pickle('./data/df_off_roll_man.pkl')
df_off_playtype_post_up.to_pickle('./data/df_off_post_up.pkl')
df_off_spot_up.to_pickle('./data/df_off_spot_up.pkl')
df_off_hand_off.to_pickle('./data/df_off_hand_off.pkl')
df_off_cut.to_pickle('./data/df_off_cut.pkl')
df_off_off_screen.to_pickle('./data/df_off_off_screen.pkl')
df_off_putbacks.to_pickle('./data/df_off_putbacks.pkl')
df_off_misc.to_pickle('./data/df_off_misc.pkl')
df_def_isolation.to_pickle('./data/df_def_isolation.pkl')
df_def_ball_handler.to_pickle('./data/df_def_ball_handler.pkl')
df_def_playtype_post_up.to_pickle('./data/df_def_post_up.pkl')
df_def_spot_up.to_pickle('./data/df_def_spot_up.pkl')
df_def_hand_off.to_pickle('./data/df_def_hand_off.pkl')
df_def_off_screen.to_pickle('./data/df_def_off_screen.pkl')
df_hustle.to_pickle('./data/df_hustle.pkl')
df_deflect.to_pickle('./data/df_deflect.pkl')